# 00 Data Exploration

Initial exploratory analysis of the legacy 125k wine dataset. Goals:
- Verify data integrity and schema
- Understand price distribution and description lengths
- Prototype lightweight heuristics for inferring wine types from free text

Run this notebook inside the project virtual environment so package versions match the pipeline.

In [1]:
import gzip
import pickle
from pathlib import Path

import numpy as np
import pandas as pd

DATA_PATH = Path("../data/00_SOURCE/wine_scraped_125k.pickle.gz")
assert DATA_PATH.exists(), f"Missing dataset: {DATA_PATH}"
with gzip.open(DATA_PATH, "rb") as fh:
    wines = pickle.load(fh)

print(f"Loaded {len(wines):,} records")
wines.head()


Loaded 125,787 records


,name,image_path,price,url,raw_html,URL_name,description
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,#95 Wine Spectator Top 100 of 2019The Brut rep...
4,Clarendon Hills Astralis Syrah 2011,/product/images/fl_progressive/qajrqr4d6ttn2pf...,149.99,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"The pride of our portfolio, Clarendon Hills As..."


In [2]:
# Price distribution summary
price = pd.to_numeric(wines["price"], errors="coerce")
price.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).to_frame(name="price_usd")


,price_usd
count,125787.000000
mean,48.548403
std,137.926125
min,0.000000
10%,11.990000
25%,15.990000
50%,23.990000
75%,46.990000
90%,89.000000
95%,129.990000


In [3]:
# Description length exploration
desc_len = wines["description"].astype(str).str.len()
desc_len.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).to_frame(name="characters")


,characters
count,125787.000000
mean,346.955647
std,256.147084
min,0.000000
10%,0.000000
25%,198.000000
50%,314.000000
75%,458.000000
90%,646.000000
95%,799.000000


In [4]:
# Simple wine type heuristic based on keyword matches
from collections import OrderedDict

KEYWORDS = OrderedDict({
    "Cabernet Sauvignon": ["cabernet sauvignon", "cabernet"],
    "Pinot Noir": ["pinot noir"],
    "Chardonnay": ["chardonnay"],
    "Red Blend": ["red blend"],
    "Merlot": ["merlot"],
    "Sauvignon Blanc": ["sauvignon blanc"],
    "Rosé": ["rosé", "rose"],
    "Sparkling": ["sparkling"],
    "Champagne": ["champagne"],
    "Zinfandel": ["zinfandel"],
    "Syrah / Shiraz": ["syrah", "shiraz"],
    "Riesling": ["riesling"],
    "Malbec": ["malbec"],
    "Tempranillo": ["tempranillo"],
    "Chianti": ["chianti"],
    "Nebbiolo": ["nebbiolo"],
    "Sangiovese": ["sangiovese"],
    "Bordeaux": ["bordeaux"],
    "Rioja": ["rioja"],
    "Port": ["port"],
})

names_lower = wines["name"].astype(str).str.lower()
descr_lower = wines["description"].astype(str).str.lower()

def match_type(name: str, desc: str) -> str:
    for label, patterns in KEYWORDS.items():
        for pattern in patterns:
            if pattern in name or pattern in desc:
                return label
    return "Other"

wines["wine_type_guess"] = [match_type(n, d) for n, d in zip(names_lower, descr_lower)]

type_counts = wines["wine_type_guess"].value_counts().head(15)
type_counts.to_frame(name="count")


,count
wine_type_guess,
Other,38256
Cabernet Sauvignon,21884
Chardonnay,13213
Pinot Noir,12784
Syrah / Shiraz,7161
Sauvignon Blanc,5541
Rosé,5247
Merlot,4656
Zinfandel,3791


In [5]:
# Price summary for the top inferred styles
price_by_type = (
    wines.dropna(subset=["wine_type_guess"])
         .assign(price_numeric=price)
         .groupby("wine_type_guess")
         .agg(count=("wine_type_guess", "size"),
              median_price=("price_numeric", "median"),
              mean_price=("price_numeric", "mean"),
              p90_price=("price_numeric", lambda s: s.quantile(0.90)))
         .query("count >= 100")
         .sort_values("median_price", ascending=False)
)
price_by_type.head(10)


,count,median_price,mean_price,p90_price
wine_type_guess,,,,
Champagne,259,54.99,123.475753,247.792
Nebbiolo,428,37.99,63.994626,104.990
Pinot Noir,12784,33.14,45.104783,79.990
Cabernet Sauvignon,21884,31.99,72.447551,139.970
Port,1894,28.74,56.440385,104.991
Other,38256,25.99,57.515706,99.990
Syrah / Shiraz,7161,24.99,39.205234,71.990
Sangiovese,756,21.99,39.172249,79.990
Bordeaux,387,21.99,123.405090,193.990


In [6]:
# High-frequency tokens in tasting notes
import re
from collections import Counter

all_words = Counter()
for desc in descr_lower:
    all_words.update(re.findall(r"[a-zA-Z']+", desc))

pd.DataFrame(all_words.most_common(25), columns=["token", "count"])


,token,count
0,and,413034
1,the,346605
2,of,259251
3,a,205240
4,with,204646
5,is,122530
6,wine,102419
7,in,92384
8,to,82977
9,this,79973
